In [1]:
import configparser
from lib import emr_cluster

In [2]:
# Read in configurations parameters 
config_filepath = 'config.cfg'
config = configparser.ConfigParser()
config.read_file(open(config_filepath))

In [4]:
# Instantiate a EMRLoader cluster class
emrCluster = emr_cluster.EMRLoader(config, keepAlive=True)

In [5]:
# define the python file to run in the step
spark_script_1 = 'test/spark-emr-TEST.py'

# define the 2 steps to be executed in the EMR cluster
# after its creation
emrCluster.custom_steps = [
    {
        'Name': 'Run spark_script_1',
        'ActionOnFailure': 'CONTINUE',
        'HadoopJarStep': {
            'Jar': 'command-runner.jar',
            'Args': [
                'spark-submit',
                '--deploy-mode', 'cluster',
                '--py-files', 's3://helptheplanet/code/{}'.format(spark_script_1),
                's3://helptheplanet/code/{}'.format(spark_script_1),
            ]
        }
    }
]

In [6]:
emrCluster.custom_steps

[{'Name': 'Run spark_script_1',
  'ActionOnFailure': 'CONTINUE',
  'HadoopJarStep': {'Jar': 'command-runner.jar',
   'Args': ['spark-submit',
    '--deploy-mode',
    'cluster',
    '--py-files',
    's3://helptheplanet/code/test/spark-emr-TEST.py',
    's3://helptheplanet/code/test/spark-emr-TEST.py']}}]

In [7]:
emrCluster.create_cluster()

ClientError: An error occurred (AccessDeniedException) when calling the RunJobFlow operation: User: arn:aws:iam::102287328023:user/uda_az is not authorized to perform: iam:PassRole on resource: arn:aws:iam::102287328023:role/EMR_DefaultRole with an explicit deny

In [10]:
emrCluster.emr_client.describe_cluster(ClusterId=emrCluster.emr_response.get("JobFlowId"))

{'Cluster': {'Id': 'j-1KKK5943XPTAJ',
  'Name': 'capstone1',
  'Status': {'State': 'WAITING',
   'StateChangeReason': {'Message': 'Cluster ready after last step completed.'},
   'Timeline': {'CreationDateTime': datetime.datetime(2021, 7, 26, 12, 21, 14, 274000, tzinfo=tzlocal()),
    'ReadyDateTime': datetime.datetime(2021, 7, 26, 12, 24, 42, 140000, tzinfo=tzlocal())}},
  'Ec2InstanceAttributes': {'Ec2KeyName': 'emr_1',
   'Ec2SubnetId': 'subnet-cd4bcdb5',
   'RequestedEc2SubnetIds': ['subnet-cd4bcdb5'],
   'Ec2AvailabilityZone': 'us-west-2b',
   'RequestedEc2AvailabilityZones': [],
   'IamInstanceProfile': 'EMR_EC2_DefaultRole',
   'EmrManagedMasterSecurityGroup': 'sg-06b806eefbf5f8c63',
   'EmrManagedSlaveSecurityGroup': 'sg-08a0fc13be045761a'},
  'InstanceCollectionType': 'INSTANCE_GROUP',
  'LogUri': 's3n://helptheplanet/emrlogs/',
  'ReleaseLabel': 'emr-5.28.0',
  'AutoTerminate': False,
  'TerminationProtected': False,
  'VisibleToAllUsers': True,
  'Applications': [{'Name': 'Sp

In [12]:
emrCluster.emr_client.terminate_job_flows(JobFlowIds=[emrCluster.emr_response.get("JobFlowId"),])

{'ResponseMetadata': {'RequestId': 'e6fb7614-2905-4b04-adf9-81c6b23819fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e6fb7614-2905-4b04-adf9-81c6b23819fb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 26 Jul 2021 12:41:54 GMT'},
  'RetryAttempts': 0}}